<a href="https://colab.research.google.com/github/alarcon7a/Langchain-con-Ollama/blob/main/local_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install fastembed langchain langchain_community PyMuPDF chromadb

## Ollama from langchain

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

llm.invoke("Hola, quien eres?")

## RAG

### Load Document

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("src/conversations.json")

In [ ]:
data_pdf = loader.load()

In [ ]:
data_pdf[1]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
docs = text_splitter.split_documents(data_pdf)

In [ ]:
docs[2]

In [ ]:
docs[3]

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from langchain_community.vectorstores import Chroma

vs = Chroma.from_documents(
    documents=docs,
    embedding=embed_model,
    persist_directory="chroma_db_dir",  # Local mode with in-memory storage only
    collection_name="stanford_report_data"
)

In [ ]:
vectorstore = Chroma(embedding_function=embed_model,
                     persist_directory="chroma_db_dir",
                     collection_name="stanford_report_data")
retriever=vectorstore.as_retriever(search_kwargs={'k': 3})


In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt_template = """Usa la siguiente información para responder a la pregunta del usuario.
Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.

Contexto: {context}
Pregunta: {question}

Solo devuelve la respuesta útil a continuación y nada más y responde siempre en español
Respuesta útil:
"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs={"prompt": prompt})

In [ ]:
response = qa.invoke({"query": "Cual es el comportamiento de los modelos fundacionales?"})

In [ ]:
response['result']

In [ ]:
response = qa.invoke({"query": "que es QLoRA?, explicamelo en detalle"})
response


In [ ]:
response['result']

In [ ]:
metadata = []
for _ in response['source_documents']:
    metadata.append((_.metadata['page'], _.metadata['file_path']))
metadata